In [1]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def e_swish_1(x):
    return K.maximum(0.0, x*(2-K.sigmoid(x)))

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [3]:
activations = [e_swish_1, e_swish_2, swish, "relu", "sigmoid", "tanh"]
names = activations[:]
for i,a in enumerate(names):
    if not isinstance(a, str):
        names[i] = a.__name__
print(names)

['e_swish_1', 'e_swish_2', 'swish', 'relu', 'sigmoid', 'tanh']


In [4]:
baseMapNum = 32
weight_decay = 1e-4

act, act_name = activations[3], names[3]
print(act_name)


model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

relu
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_____

In [5]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [6]:
#training
batch_size = 128
epochs=25
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep75'+act_name+'.h5')

Epoch 1/75
390/390 [==============================] - 148s - loss: 1.9521 - acc: 0.4123 - val_loss: 1.4238 - val_acc: 0.5036
Epoch 2/75
390/390 [==============================] - 143s - loss: 1.3625 - acc: 0.5678 - val_loss: 1.1092 - val_acc: 0.6505
Epoch 3/75
390/390 [==============================] - 143s - loss: 1.2461 - acc: 0.6244 - val_loss: 1.3889 - val_acc: 0.6417
Epoch 4/75
390/390 [==============================] - 142s - loss: 1.1942 - acc: 0.6501 - val_loss: 0.9129 - val_acc: 0.7195
Epoch 5/75
390/390 [==============================] - 142s - loss: 1.1308 - acc: 0.6699 - val_loss: 0.9396 - val_acc: 0.7028
Epoch 6/75
390/390 [==============================] - 143s - loss: 1.1085 - acc: 0.6817 - val_loss: 0.7849 - val_acc: 0.7551
Epoch 7/75
390/390 [==============================] - 143s - loss: 1.0347 - acc: 0.7017 - val_loss: 0.7559 - val_acc: 0.7657
Epoch 8/75
390/390 [==============================] - 143s - loss: 1.0564 - acc: 0.7066 - val_loss: 0.7671 - val_acc: 0.7661


390/390 [==============================] - 142s - loss: 0.6409 - acc: 0.8338 - val_loss: 0.5420 - val_acc: 0.8654
Epoch 67/75
390/390 [==============================] - 142s - loss: 0.6370 - acc: 0.8370 - val_loss: 0.5516 - val_acc: 0.8673
Epoch 68/75
390/390 [==============================] - 143s - loss: 0.6430 - acc: 0.8338 - val_loss: 0.5433 - val_acc: 0.8667
Epoch 69/75
390/390 [==============================] - 142s - loss: 0.6279 - acc: 0.8385 - val_loss: 0.5544 - val_acc: 0.8691
Epoch 70/75
390/390 [==============================] - 142s - loss: 0.6452 - acc: 0.8353 - val_loss: 0.5369 - val_acc: 0.8657
Epoch 71/75
390/390 [==============================] - 143s - loss: 0.6315 - acc: 0.8380 - val_loss: 0.5711 - val_acc: 0.8565
Epoch 72/75
390/390 [==============================] - 142s - loss: 0.6306 - acc: 0.8386 - val_loss: 0.5519 - val_acc: 0.8592
Epoch 73/75
390/390 [==============================] - 142s - loss: 0.6337 - acc: 0.8370 - val_loss: 0.5468 - val_acc: 0.8642
Epoc

In [7]:
opt_rms = keras.optimizers.rmsprop(lr=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep100'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 145s - loss: 0.5859 - acc: 0.8509 - val_loss: 0.4887 - val_acc: 0.8847
Epoch 2/25
390/390 [==============================] - 145s - loss: 0.5720 - acc: 0.8523 - val_loss: 0.4691 - val_acc: 0.8867
Epoch 3/25
390/390 [==============================] - 142s - loss: 0.5620 - acc: 0.8577 - val_loss: 0.4926 - val_acc: 0.8810
Epoch 4/25
390/390 [==============================] - 142s - loss: 0.5546 - acc: 0.8589 - val_loss: 0.4842 - val_acc: 0.8850
Epoch 5/25
390/390 [==============================] - 142s - loss: 0.5574 - acc: 0.8584 - val_loss: 0.4627 - val_acc: 0.8910
Epoch 6/25
390/390 [==============================] - 142s - loss: 0.5548 - acc: 0.8590 - val_loss: 0.4836 - val_acc: 0.8837
Epoch 7/25
390/390 [==============================] - 142s - loss: 0.5452 - acc: 0.8579 - val_loss: 0.4590 - val_acc: 0.8911
Epoch 8/25
390/390 [==============================] - 142s - loss: 0.5433 - acc: 0.8595 - val_loss: 0.4612 - val_acc: 0.8892


In [8]:
opt_rms = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep125'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 143s - loss: 0.4817 - acc: 0.8753 - val_loss: 0.4213 - val_acc: 0.8949
Epoch 2/25
390/390 [==============================] - 142s - loss: 0.4748 - acc: 0.8758 - val_loss: 0.4332 - val_acc: 0.8945
Epoch 3/25
390/390 [==============================] - 143s - loss: 0.4674 - acc: 0.8776 - val_loss: 0.4215 - val_acc: 0.8944
Epoch 4/25
390/390 [==============================] - 154s - loss: 0.4725 - acc: 0.8770 - val_loss: 0.4211 - val_acc: 0.8950
Epoch 5/25
390/390 [==============================] - 148s - loss: 0.4647 - acc: 0.8782 - val_loss: 0.4157 - val_acc: 0.8954
Epoch 6/25
390/390 [==============================] - 143s - loss: 0.4689 - acc: 0.8768 - val_loss: 0.4240 - val_acc: 0.8955
Epoch 7/25
390/390 [==============================] - 143s - loss: 0.4647 - acc: 0.8786 - val_loss: 0.4261 - val_acc: 0.8942
Epoch 8/25
390/390 [==============================] - 142s - loss: 0.4605 - acc: 0.8774 - val_loss: 0.4337 - val_acc: 0.8908


In [9]:
#testing - no kaggle eval
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

 9984/10000 [============================>.] - ETA: 0s
Test result: 90.120 loss: 0.402
